### Walkthrough of model deployment as ML web service on Kubernetes

In [20]:
from yaml import load, Loader
import os
import requests
import json

Load configuration

In [8]:
with open('config.yaml','r') as config_file:
    config = load(config_file, Loader=Loader)

docker_registry = config['DOCKER_REGISTRY']
service_name = config['SERVICE_NAME']
api_version = config['API_VERSION']
model_repo = '..\experimentation\models'

Copy latest model to deployment directory

In [16]:
latest_model = sorted(os.listdir(model_repo))[-1]
latest_model_path = os.path.join(model_repo,latest_model)

!copy {latest_model_path} .

        1 file(s) copied.


Build the docker image

In [17]:
tag = f'{docker_registry}/{service_name}:{api_version}'
!docker build -t {tag} .

#1 [internal] load build definition from Dockerfile
#1 sha256:fbdc11ca5b91d337a697cd95f2fe04894caf8873ee5f0ab625a505ec197667db
#1 transferring dockerfile: 225B done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:85698ddf877abbf5684f60088013dbbafff4b1241ea04b9bf874a688fde00629
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.9-slim
#3 sha256:3425157df499c84dd49181e5611a11caeed16adf15a5ddbcfa4c3002c56d3d27
#3 DONE 0.8s

#4 [1/5] FROM docker.io/library/python:3.9-slim@sha256:d67e4b3e185208a010e0d06dd1f655292dd92c5dd08a64b1c59a0acbd387b1e9
#4 sha256:50f4b98c190db8b5a8550f1967c6924d86d16143ba5d31809975402465cf3079
#4 DONE 0.0s

#5 [2/5] RUN mkdir /app
#5 sha256:e1f3069cec88a973a3eae25ca94e7435bb7fb26846c5bcefeb9c047006d151f5
#5 CACHED

#6 [internal] load build context
#6 sha256:b020678d324f37b313fe9ba90751166cdce3e49c32fa05240a6109d70b336a4e
#6 transferring context: 3.43kB done
#6 DONE 0.0s

#7 [3/5] COPY config.yaml api.py 

Push the image to Docker hub

In [ ]:
!docker push {tag}

Deploy the image to Kubernetes using Helm

In [18]:
!helm upgrade --install mlwebservice-titanic helm-ml-serving

Release "mlwebservice-titanic" does not exist. Installing it now.
NAME: mlwebservice-titanic
LAST DEPLOYED: Sat Dec 11 16:17:00 2021
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None


Test the web service

In [25]:
# example of an input that should yield 0
X1 = [-0.11338264511659583,-1.0524307127141423, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]

# example of an input that should yield 1
X2 = [-1.632430989759686,0.231855615029561,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0]

# define base URL (localhost + flask port)
url = f"http://127.0.0.1:5000/{service_name}/v{api_version}/predict"
body={'X':X2}
headers = {"Content-Type": "application/json"}

In [26]:
# send a get request to flask api
response = requests.post(url=url, data=json.dumps(body), headers=headers)
print(response.json()) 

{'prediction': 1}
